## 1) NLP에서의 pre-training
* 사전 훈련된 단어 이베딩이 모든 nlp 실무자의 도구 상자에서 사전 훈련된 언어 모델로 대체되는 것은 시간 문제이다
* 사전 훈련된 워드 임베딩부터 ELMo, 트랜스포머에 이르기까지 어떻게 발전?

### 1. 사전 훈련된 워드 임베딩
* Word2Vec, FastText, GloVE 같은 워드 임베딩 방법론들을 학습
* 어떤 테스크 수행 시 임베딩 사용 방법 2가지
1. 임베딩 층을 랜덤 초기화하여 처음부터 학습
2. 방대한 데이터로 Word2Vec과 같은 임베딩 알고리즘으로 이미 학습된 임베딩 벡터들을 가져와 사용하는 것
* 문제점 존재
  * 하나의 단어가 하나의 벡터값으로 맵핑되므로 **문맥을 고려하지 못한다**는 단점
    * 다의어나 동음이의어 구분 못함
    * 사전 훈련된 언어 모델을 사용하므로서 극복할 수 있었음

### 2. 사전 훈련된 언어 모델
* 2015년 구글의 'Semi-supervised sequence learning'이라는 논문에서 LSTM 언어 모델을 학습하고 나서 이렇게 학습한 LSTM을 텍스트 분류에 추가 학습하는 방법 제안
  1. 우선 LSTM 언어 모델 학습
    * 언어 모델은 주어진 텍스트로부터 이전 단어들로부터 다음 단어를 예측하도록 학습하므로 기본적으로 별도의 레이블이 부착되지 않은 텍스트 데이터로도 학습 가능
    * 이렇게 레이블이 없는 데이터로 학습된 LSTM과 가중치가 랜덤으로 초기화된 LSTM 두 가지를 두고, IMDB 리뷰 데이터를 학습하여 정확도를 테스트한다면 전자의 LSTM이 좀 더 좋은 성능을 얻음
    * 방대한 텍스트로 LSTM 언어 모델을 학습해두고, 이러한 언어 모델을 다른 테스크에 사용하는 방법으로는 앞서 학습한 ELMo와 같은 아이디어도 존재

* **ELMo**: 순방향 언어 모델과 역방향 언어 모델을 각각 따로 학습시킨 후, 이렇게 사전 학습된 언어 모델로부터 임베딩 값을 얻는다
  * 이 임베딩은 문장에 따라서 의미는 다르지만 소리가 같은 단어들이라고 하더라도 임베딩 벡터값이 달라지므로, Word2Vec이나 GloVe 등과 같은 워드 임베딩 문제 해결 가능

* 트랜스포머 등장 후, 언어 모델 학습 시 LSTM이 아닌 트랜스포머를 사용하는 시도 등장
  * 트랜스포머의 디코더는 LSTM 언어 모델처럼 순차적으로 이전 단어들로부터 다음 단어를 예측
    * Open AI는 트랜스포머 디코더로 총 12개의 층을 쌓은 후, 방대한 텍스트 데이터로부터 언어 모델 GPT-1을 만듦
    * GPT-1에 여러 다양한 테스크를 위해 추가 학습을 시켰을 때, 높은 성능을 얻을 수 있음을 보여줌

* 이제 NLP의 트렌드는 사전 훈련된 언어 모델을 만들어, 이를 다른 태스크에 추가 학습하여 높은 성능을 얻는 것으로 넘어오게 되었음
  * 이를 더 잘 하기 위해 기존의 언어 모델의 구조를 바꿔보고자 하는 시도가 생겨남
    * 언어의 문맥이라는 것은 실제로 양방향이 존재. 하지만 이전 단어들로부터 다음 단어를 예측하는 언어 모델의 특성으로 인해 양방향을 사용할 수 없으므로, 그 대안으로 ELMo에서는 단방향과 역방향으로 언어 모델을 분리해서 학습하는 방법 사용
    * 2018년에는 언어 모델의 양방향 학습을 위해 새로운 구조의 언어 모델 탄생, 이는 마스크드 언어 모델

### 3. 마스크드 언어 모델(Masked Language Model)
* 정의: 입력 텍스트의 단어 집합의 15%의 단어를 랜덤으로 마스킹함. 
  * 마스킹: 원래의 단어가 무엇이었는지 모르게 함
  * 인공 신경망에 이렇게 마스킹된 단어들을 예측하도록 함
    * 문장 중간에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측하는 식으로
    

## 2) 버트(Bidirectional Encoder Representations from Transformers, BERT)
* 2018년 구글이 공개한 사전 훈련된 모델

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 61.6 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### 1. BERT의 개요
* BERT는 트랜스포머를 이용하여 구현되었으며, 위키피디아(25억 단어)와 BooksCorpus(8억 단어)와 같은 레이블이 없는 텍스트 데이터로 사전 훈련된 언어 모델
* 높은 성능 얻을 수 있었던 이유?
  * 레이블이 없는 방대한 데이터로 사전 훈련된 모델을 가지고, 레이블이 있는 다른 작업(Task)에서 추가 훈련과 함께 하이퍼파라미터를 재조정하여 이 모델을 사용하면 성능이 높게 나오는 기존의 사례들을 참고하였기 때문
    * **파인 튜닝(Fine-tuning)**: 다른 작업에 대해서 파라미터 재조정을 위한 추가 훈련 과정

### 2. BERT의 크기
* BERT의 기본 구조는 트랜스포머의 인코더를 쌓아올린 구조
  * Base 버전에서는 총 12개, Large 버전에서는 총 24개를 쌓음
  * Large 버전은 Base 버전보다 d_model의 크기나 셀프 어텐션 헤드(Self Attention Heads)의 수가 더 큼
    * 트랜스포머 인코더 층의 수를 L, d_model의 크기를 D, 셀프 어텐션 헤드의 수를 A라고 했을 때 각각의 크기는 아래와 같음
      * BERT-Base: L=12, D=768, A=12 : 110M개의 파라미터
      * BERT-Large: L=24, D=1024, A=16 : 340M개의 파라미터
    * BERT-base는 BERT보다 앞서 등장한 Open AI GPT-1과 하이퍼파라미터가 동일한데, 이는 BERT 연구진이 직접적으로 GPT-1과 성능을 비교하기 위해서 동등한 크기로 설계함
    * BERT-Large는 최대 성능을 보여주기 위해 만들어진 모델


### 3. BERT의 문맥을 반영한 임베딩(Contextual Embedding)
* BERT는 ELMo나 GPT-1과 마찬가지로 문맥을 반영한 임베딩 사용
* BERT의 입력: 임베딩층을 지난 임베딩 벡터들
  * d_model: 768, 모든 단어들은 768차원의 임베딩 벡터가 되어 BERT의 입력으로 사용됨
  * BERT는 내부적인 연산을 거친 후, 동일하게 각 단어에 대해서 768차원의 벡터 출력
* BERT의 연산을 거친 후의 출력 임베딩: 문장의 문맥을 모두 참고한 문맥을 반영한 임베딩
  * 하나의 단어가 모든 단어를 참고하는 연산은 사실 BERT의 12개의 층에서 전부 이루어지는 연산
  * 그리고 이를 12개의 층을 지난 후에 최종적으로 출력 임베딩을 얻게 되는 것
  * BERT의 첫번째 층의 출력 임베딩은 BERT의 두번째 층에서는 입력 임베딩이 됨
* 그렇다면 BERT는 어떻게 모든 단어들을 참고하여 문맥을 반영한 출력 임베딩을 얻게 되는 것?
  * 답: **셀프 어텐션**
  * BERT는 기본적으로 트랜스포머 인코더를 12번 쌓은 것이므로 내부적으로 각 층마다 멀티 헤드 셀프 어텐션과 포지션 와이즈 피드 포워드 신경망 수행


### 4. BERT의 서브워드 토크나이저: WordPiece
* BERT는 단어보다 더 작은 단위로 쪼개는 서브워드 토크나이저를 사용
* BERT가 사용한 토크나니저는 WordPiece 토크나이저로 14챕터에서 공부한 바이트 페어 인코딩(Byte Pair Encoding, BPE)의 유사 알고리즘
  * 동작 방식은 BPE와 조금 다르지만, 글자로부터 서브워드들을 병합해가는 방식으로 최종 단어 집합(Vocabulary)를 만드는 것은 BPE와 유사
* 서브워드 토크아니저: 기본적으로 자주 등장하는 단어는 그대로 단어 집합에 추가하지만, 자주 등장하지 않는 단어의 경우에는 더 작은 단위인 서브워드로 분리되어 서브워드들이 단어 집합에 추가된다는 아이디어
  * 이렇게 단어 집합이 만들어지고 나면, 이 단어 집합을 기반으로 토큰화 수행
  * 이는 대표적인 서브워드 토크나이저 패키지인 SentencePiece 실습을 통해 이해한 내용
  * BERT의 서브워드 토크나이저도 이와 마찬가지로 동작
* BERT에서 토큰화를 수행하는 방식
  * 준비물: 이미 훈련 데이터로붙 만들어진 단어 집합
    * 1. 토큰이 단어 집합에 존재한다. => 해당 토큰을 분리하지 않는다.
    * 2. 토큰이 단어 집합에 존재하지 않는다. => 해당 토큰을 서브워드로 분리한다. => 해당 토큰의 첫번째 서브워드를 제외한 나머지 서브워드들은 앞에 "##"를 붙인 것을 토큰으로 한다.

  * embeddings: 서브워드 토크나이저가 아니라면 OOV 문제가 발생. 하지만 서브워드 토크나이저의 경우에는 해당 단어가 단어 집합에 존재하지 않았다고 해서, 서브워드 또한 존재하지 않는다는 의미는 아니므로 해당 단어를 더 쪼개려고 시도
    * 만약, BERT의 단어 집합에 em, ##bed, ##ding, #s라는 서브 워드들이 존재한다면, embeddings는 em, ##bed, ##ding, #s로 분리됨
      * ##는 서브워드들은 단어의 중간부터 등장하는 서브워드라는 것을 알려주기 위해서 단어 집합 생성 시 표시해둔 기호. 이런 표시가 있어야만 쉽게 embeddings로 복원 가능



In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") #Bert-base의 토크나이저
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [ ]:
# BERT의 단어 집합에 특정 단어가 있는지 조회
print(tokenizer.vocab['here'])

NameError: ignored

In [ ]:
# 단어 임베딩 조회
print(tokenizer.vocab['embeddings'])

# 해당 단어가 존재하지 않는다는 의미에서 KeyError가 발생

KeyError: ignored

In [ ]:
print(tokenizer.vocab['em'])

7861


In [ ]:
print(tokenizer.vocab['##bed'])

8270


In [ ]:
print(tokenizer.vocab['##ding'])

4667


In [ ]:
print(tokenizer.vocab['##s'])

2015


In [ ]:
# 이제는 BERT의 단어 집합 전체를 불러와서 살펴봄
# BERT의 단어 집합을 vocabulary.txt에 저장

with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')

In [ ]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [ ]:
print('단어 집합의 크기 :',len(df))

단어 집합의 크기 : 30522


In [ ]:
# 정수로부터 맵핑된 단어 확인 가능
df.loc[4667].values[0]

'##ding'

In [ ]:
df.loc[102].values[0]

'[SEP]'

### 5. 포지션 임베딩(Position Embedding)
* 트랜스포머에서는 포지셔널 인코딩이라는 방법을 통해서 단어의 위치 정보 표현
  * 포지셔널 인코딩: 사인 함수와 코사인 함수를 사용하여 위치에 따라 다른 값을 가지는 행렬을 만들어 이를 단어 벡터들과 더하는 방법
  * BERT에서는 이와 유사하지만, 위치 정보를 사인 함수와 코사인 함수로 만드는 것이 아닌, **학습을 통해서 얻는 포지션 임베딩**이라는 방법 사용

* WordPiece Embedding: 우리가 이미 알고 있는 단어 임베딩. 실직적인 입력.
  * 이 입력에 포지션 임베딩을 통해서 **위치 정보 더해주기**
  * 포지션 임베딩 아이디어 굉장히 간단: **위치 정보를 위한 임베딩 층(Embedding layer)을 하나 더 사용**
  * 문장의 길이가 4라면 4개의 포지션 임베딩 벡터를 학습
  * 그리고 BERT의 입력마다 다음과 같이 포지션 임베딩 벡터를 더해주는 것
    * 첫번째 단어의 임베딩 벡터 + 0번 포지션 임베딩 벡터
    * 두번째 단어의 임베딩 벡터 + 1번 포지션 임베딩 벡터
    * 세번째 단어의 임베딩 벡터 + 2번 포지션 임베딩 벡터
    * 네번째 단어의 임베딩 벡터 + 3번 포지션 임베딩 벡터
  * 실제 BERT에서는 문장의 최대 길이를 512. 총 512개의 포지션 임베딩 벡터가 학습됨
  * 결론적으로 현재 설명한 내용을 기준으로는 BERT에서는 총 두 개의 임베딩 층이 사용됨
    * 단어 집합의 크기가 30,522개인 단어 벡터를 위한 임베딩 층
    * 문장의 최대 길이가 512개의 포지션 벡터를 위한 임베딩 층
* 사실 BERT는 세그먼트 임베딩이라는 1개의 임베딩 층 더 사용
### 6. BERT의 Pre-training
* ELMo: 정방향 LSTM과 역방향 LSTM을 각각 훈련시키는 방식으로 양방향 언어 모델 만듦
* GPT-1: 트랜스포머의 디코더를 이전 단어들로부터 다음 단어를 예측하는 방식으로 단방향 언어 모델 만듦
* BERT: 단방향으로 설계된 GPT-1과는 달리, 화살표가 양방향으로 뻗어나감. 이는 Masked Language Model을 통해 양방향성을 얻었기 때문.
  * BERT의 사전 훈련 방법은 크게 두 가지: 
    1. **마스크드 언어 모델 (Masked Language Model)**
    2. **다음 문장 예측 (Next sentence prediction, NSP)**

## 1) Masked Language Model(MLM)
* BERT는 사전 훈련을 위해서 인공 신경망의 입력으로 들어가는 입력 텍스트의 15%의 단어를 랜덤으로 마스킹
  * 그리고 인공 신경망에게 이 가려진 단어들(Masked words)를 예측하도록 함
  * 중간에 단어들에 구멍을 뚫어놓고, 구멍에 들어갈 단어들을 예측하게 하는 식
* 랜덤으로 선택된 15%의 단어들은 다음과 같은 비율로 규칙이 적용됨
  * 80%의 단어들은 [MASK]로 변경됨
  * 10%의 단어들은 랜덤으로 단어가 변경됨
  * 10%의 단어들은 동일하게 둠
* 이렇게 하는 이유: 
  * [MASK]만 사용할 경우에는 [MASK] 토큰이 파인 튜닝 단계에서는 나타나지 않으므로 사전 학습 단계와 파인 튜닝 단계에서의 불일치가 발생하는 문제
    * 이 문제를 완화하기 위해서 랜덤으로 선택된 15%의 단어들의 모든 토큰을 [MASK]로 사용하지 않음
    * 이를 전체 단어 관점에서 그래프를 통해 정리하면 다음과 같음
      * 그래프로 정리 시, 85% 미사용, 12% [MASK]로 변경 후 예측, 1.5% 랜덤으로 변경 후 예측, 1.5% 미변경 후 예측

## 2) 다음 문장 예측 (Next Sentence Prediction, NSP)
* 두 개의 문장을 준 후에 이 문장이 이어지는 문장인지 아닌지를 맞추는 방식으로 훈련. 이를 위해서 50:50 비율로 실제 이어지는 두 개의 문장과 랜덤으로 이어붙인 두 개의 문장을 주고 훈련
  * 이를 각각 Sentence A와 Sentence B라고 하였을 때, 다음의 예는 문장의 연속성을 확인한 경우와 그렇지 않은 경우 보여줌
* BERT의 입력으로 넣을 때는 [SEP]라는 특별 토큰을 사용해서 문장 구분
  * 첫번째 문장 끝에 [SEP] 토큰을 넣고, 두번째 문장이 끝나면 역시 [SEP] 토큰을 붙여줌
  * 그리고 이 두 문장이 실제 이어지는 문장인지 아닌지를 [CLS] 토큰의 위치의 출력층에서 이진 분류 문제를 풀도록 함
  * [CLS] 토큰은 BERT가 분류 문제를 풀기 위해 추가된 특별 토큰
* 그리고 마스크드 언어 모델과 다음 문장 예측은 따로 학습하는 것이 아닌 loss를 합하여 학습이 동시에 이루어짐
* BERT가 언어 모델 외에도 다음 문장 예측이라는 테스크를 학습하는 이유는 BERT가 풀고자 하는 테스크 중에서는 QA(Question Answering)과 NLI(Natural Language Inference)와 같이 두 문장의 관계를 이해하는 것이 중요한 태스크들이 있기 때문
  


### 7. 세그먼트 임베딩 (Segement Embedding)
* BERT는 QA 등과 같은 두 개의 문장 입력이 필요한 태스크를 풀기도 함
* 문장 구분을 위해서는 BERT는 세그먼트 임베딩이라는 또 다른 임베딩층(Embedding Layer)를 사용
  * 첫 번째 문장에는 Sentence 0 임베딩, 두번째 문장에는 Sentence 1 임베딩을 더해주는 방식이며 임베딩 벡터는 두 개만 사용됨 
* 결론적으로 BERT는 총 3개의 임베딩 층 사용됨
  * WordPiece Embedding : 실질적인 입력이 되는 워드 임베딩. 임베딩 벡터의 종류는 단어 집합의 크기로 30,522개.
  * Position Embedding : 위치 정보를 학습하기 위한 임베딩. 임베딩 벡터의 종류는 문장의 최대 길이인 512개.
  * Segment Embedding : 두 개의 문장을 구분하기 위한 임베딩. 임베딩 벡터의 종류는 문장의 최대 개수인 2개.
* 주의할 점: 많은 문헌에서 BERT가 문장 중간의 [SEP] 토큰과 두 종류의 세그먼트 임베딩을 통해서 두 개의 문장을 구분하여 입력 받을 수 있다고 설명하고 있지만, 여기서 BERT에 두 개의 문장이 들어간다는 표현에서의 문장이라는 것은 실제 우리가 알고 있는 문장의 단위는 아님
  * 예를 들어 QA 문제를 푸는 경우에는 [SEP]와 세그먼트 임베딩을 기준으로 구분되는 [질문(Question), 본문(Paragraph)] 두 종류의 텍스트를 입력받지만, Paragraph 1개는 실제로 다수의 문장으로 구성될 수 있음
  * 즉, [SEP]와 세그먼트 임베딩으로 구분되는 BERT의 입력에서의 두 개의 문장은 실제로는 두 종류의 텍스트, 두 개의 문서일 수 있음
* BERT가 두 개의 문장을 입력받을 필요가 없는 경우: ex. 네이버 영화 리뷰나 IMDB 리뷰 분류와 같은 감성 분류 테스크에서는 한 개의 문서에 대해서만 분류를 하는 것이므로, 이 경우에는 BERT의 전체 입력에 Sentence 0 임베딩만을 더해줌. 아래에 설명할 1)과 2) 파인튜닝 윻여이 그 예

### 8. BERT를 Fine-tuning하기
* 사전 학습된 BERT에 우리가 풀고자 하는 테스크의 데이터를 추가로 학습 시켜서 테스트하는 단계인 파인 튜닝 단계에 대해서 알아봄
* 실질적으로 테스크에 BERT를 사용하는 단계
#### 1) 하나의 텍스트에 대한 텍스트 분류 유형(Single Text Classification)
* 영화 리뷰 감성 분류, 로이터 뉴스 분류 등과 같이 입력된 문서에 대해서 분류를 하는 유형
* 문서의 시작에 [CLS]라는 토큰 입력
* 앞서 사전 훈련 단계에서 다음 문장 예측을 설명할 때, [CLS] 토큰은 BERT가 분류 문제를 풀기 위한 특별 토큰이라고 언급한 바 있음
  * 이는 BERT를 실질적으로 사용하는 단계인 파인 튜닝 단계에서도 마찬가지
* 텍스트 분류 문제를 풀기 위해서 [CLS] 토큰의 위치의 출력층에서 밀집층(Dense layer) 또는 같은 이름으로는 완전 연결층(fully-connected layer)이라고 불리는 층들을 추가하여 분류에 대한 예측을 하게 됨
#### 2) 하나의 텍스트에 대한 태깅 작업(Tagging)
* BERT를 사용하는 두 번째 유형은 태깅 작업
* 13챕터에서 RNN 계열의 신경망들을 이용해서 풀었던 테스크
* 대표적으로 문장의 각 단어에 품사를 태깅하는 품사 태깅 작업과 개체를 태깅하는 개체명 인식 작업이 있음
* 출력층에서는 입력 텍스트의 각 토큰의 위치에 밀집층을 사용하여 분류에 대한 예측을 하게 됨

#### 3) 텍스트의 쌍에 대한 분류 또는 회귀 문제(Text Pair Classification or Regression)
* BERT는 텍스트의 쌍을 입력으로 받는 테스크도 풀 수 있음
* 대표적인 테스크로 자연어 추론(Natural Language Inference)
  * 두 문장이 주어졌을 때, 하나의 문장이 다른 문장과 논리적으로 어떤 관계에 엤는지를 분류
  * 유형: 모순 관계(contradiction), 함의 관계(entailment), 중립 관계(neutral)
* 텍스트의 쌍을 입력 받는 이러한 테스크의 경우에는 입력 텍스트가 1개가 아니므로, 텍스트 사이에 [SEP] 토큰을 집어넣고, Sentence 0 임베딩과 Sentence 1 임베딩이라는 두 종류의 세그먼트 임베딩을 모두 사용하여 문서 구분

#### 4) 질의 응답 Question Answering
* 텍스트의 쌍을 입력으로 받음
* 질문과 본문이라는 두 개의 텍스트의 쌍 입력
* 대표적인 데이터셋: SQuAD(Stanford Question Answering Dataset) v1.1
  * 질문과 본문을 입력받으면, 본문의 일부분을 추출해서 질문에 답변하는 것

### 9. 기타
  * 두 문장 Sentence A and B 합한 길이. 최대 입력 길이는 512.
  * 100만 step 훈련. 33억 단어 코퍼스에 대해 40 에포크 학습
  * 옵티아미저: Adam
  * 학습률: 10^-4
  * 가중치 감소: L2 정규화로 0.01 적용
  * 드롭 아웃: 모든 레이어에 대해서 0.1 적용
  * 활성화 함수: relu가 아닌 gelu 함수
  * 배치 크기: 256

### 10. 어텐션 마스크(Attention Mask)
* BERT를 실제로 실습하게 되면 어텐션 마스크라는 시쿠너스 입력이 추가로 필요
* 어텐션 마스크: BERT가 어텐션 연산을 할 때, 불필요하게 패딩 토큰에 대해서 어텐션을 하지 않도록 실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 입력
  * 이 값은 0과 1 두 가지 값을 가지는데, 숫자 1은 해당 토큰은 실제 단어이므로 마스킹을 하지 않는다는 의미
  * 숫자 0은 해당 토큰은 패딩 토큰이므로 마스킹을 한다
  

## 3) 구글 BERT의 Masked Language Model 실습

In [ ]:
!pip install transformers

### 1. 마스크드 언어 모델
* transformers 패키지를 사용하여 모델과 토크나이저 로드
* BERT는 이미 누군가가 학습해둔 모델을 사용하는 것
  * 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 함
    * 예를 들어 A라는 이름의 BERT 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없음
    * A라는 BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 단어와 맵핑되는 정수 정보 자체가 다르기 때문

In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

* TFBertForMaskedLM.from_pretrained('BERT 모델 이름')을 넣으면 [MASK]라고 되어있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드함
* 다시 말해서 BERT를 마스크드 언어 모델 형태로 로드함

* AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드함


In [ ]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

### 2. BERT의 입력
* 마스크드 언어 모델의 예측 결과를 보기 위해서 bert-large-uncased의 토크나이저를 사용하여 해당 문장을 정수 인코딩

In [ ]:
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [ ]:
# 토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인 가능
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [ ]:
# 토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과 확인
print(inputs['token_type_ids'])

# 현재 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장의 길이만큼의 0 시퀀스를 얻음

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [ ]:
# 토크나이저로 변환된 결과에서 attention_mask를 통해서 
# 실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크 확인 가능

print(inputs['attention_mask'])

# 현재 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻음
# 만약 뒤에 패딩이 있었따면 패딩이 시작되는 구간부터는 0의 시퀀스가 나오게 되지만,
# 여기에서는 해당되지 않음

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


### 3. [MASK] 토큰 예측하기
* FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줌
* 우선 앞서 불러온 모델과 토크나이저 저장

In [ ]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [ ]:
# 입력 문장으로부터 [MASK]의 위치에 들어갈 수 있는 상위 5개의 후보 단어 출력
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621122598648071,
  'sequence': 'soccer is a really fun sport.',
  'token': 4368,
  'token_str': 'sport'},
 {'score': 0.20341984927654266,
  'sequence': 'soccer is a really fun game.',
  'token': 2208,
  'token_str': 'game'},
 {'score': 0.012208555825054646,
  'sequence': 'soccer is a really fun thing.',
  'token': 2518,
  'token_str': 'thing'},
 {'score': 0.0018630236154422164,
  'sequence': 'soccer is a really fun activity.',
  'token': 4023,
  'token_str': 'activity'},
 {'score': 0.0013354856055229902,
  'sequence': 'soccer is a really fun field.',
  'token': 2492,
  'token_str': 'field'}]

In [ ]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.2562902271747589,
  'sequence': 'the avengers is a really fun show.',
  'token': 2265,
  'token_str': 'show'},
 {'score': 0.17284105718135834,
  'sequence': 'the avengers is a really fun movie.',
  'token': 3185,
  'token_str': 'movie'},
 {'score': 0.11107684671878815,
  'sequence': 'the avengers is a really fun story.',
  'token': 2466,
  'token_str': 'story'},
 {'score': 0.07248969376087189,
  'sequence': 'the avengers is a really fun series.',
  'token': 2186,
  'token_str': 'series'},
 {'score': 0.07046617567539215,
  'sequence': 'the avengers is a really fun film.',
  'token': 2143,
  'token_str': 'film'}]

In [ ]:
pip('I went to [MASK] this morning.')

[{'score': 0.3573073744773865,
  'sequence': 'i went to work this morning.',
  'token': 2147,
  'token_str': 'work'},
 {'score': 0.23304423689842224,
  'sequence': 'i went to bed this morning.',
  'token': 2793,
  'token_str': 'bed'},
 {'score': 0.1284504532814026,
  'sequence': 'i went to school this morning.',
  'token': 2082,
  'token_str': 'school'},
 {'score': 0.06230573356151581,
  'sequence': 'i went to sleep this morning.',
  'token': 3637,
  'token_str': 'sleep'},
 {'score': 0.046952590346336365,
  'sequence': 'i went to class this morning.',
  'token': 2465,
  'token_str': 'class'}]

## 4) 한국어 BERT의 마스크드 언어 모델 실습


In [3]:
! pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
     |████████████████████████████████| 3.3 MB 21.0 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#### 1. 마스크드 언어 모델
* transformers 패키지를 사용하여 모델과 토크나이저를 로드
* BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계
  * A라는 이름의 BERT를 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없음
  * A라는 BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면에, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 단어와 맵핑되는 정수 정보 자체가 다르기 때문

* klue/bert-base는 대표적인 한국어 BERT
* klue/bert-base의 마스크드 언어 모델과 klue/bert-base의 토크나이저를 로드

In [4]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

* TFBertForMaskedLM.from_pretrained('BERT 모델 이름')을 넣으면 [MASK]라고 되어있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드
* 다시 말해서 BERT를 마스크드 언어 모델 형태로 로드
* from_pt = True는 해당 모델이 기존에는 텐서플로우가 아니라 파이토치로 학습된 모델이었지만 이를 텐서플로우에서 사용하겠다라는 의미
* AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

In [6]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

### 2. BERT의 입력
* '축구는 정말 재미있는 [MASK]다'라는 임의의 문장이 있음
* 이를 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측함
* 마스크드 언어 모델의 예측 결과를 보기 위해서 klue/bert-base의 토크나이저를 사용하여 해당 문장을 정수 인코딩

In [7]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')
print(inputs['input_ids'])

# 토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [8]:
print(inputs['token_type_ids'])

# 토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 학인

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [9]:
print(inputs['attention_mask'])

# 토크나이저로 변환된 결과에서 attention_mask를 통해서 실제 단어와 패딩 토큰 구분하는 용도인 어텐션 마스크 확인 가능

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


* 현재 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻음
* 만약 뒤에 패딩이 있었다면 패딩이 시작되는 구간부터는 0의 시퀀스가 나오게 되지만, 여기서는 해당되지 않음
* 좀 더 다양한 패턴의 입력은 뒤의 텍스트 분류, 개체명 인식, 질의 응답 실습에서 이어서 봄

### 3. [MASK] 토큰 예측하기
* FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줌
  * 우선 앞서 불러온 모델과 토크나이저를 지정해줌
  

In [12]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [13]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.896351158618927,
  'sequence': '축구는 정말 재미있는 스포츠 다.',
  'token': 4559,
  'token_str': '스포츠'},
 {'score': 0.025957664474844933,
  'sequence': '축구는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.010033938102424145,
  'sequence': '축구는 정말 재미있는 경기 다.',
  'token': 3682,
  'token_str': '경기'},
 {'score': 0.00792439840734005,
  'sequence': '축구는 정말 재미있는 축구 다.',
  'token': 4713,
  'token_str': '축구'},
 {'score': 0.007844222709536552,
  'sequence': '축구는 정말 재미있는 놀이 다.',
  'token': 5845,
  'token_str': '놀이'}]

In [14]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.83824223279953,
  'sequence': '어벤져스는 정말 재미있는 영화 다.',
  'token': 3771,
  'token_str': '영화'},
 {'score': 0.02827564999461174,
  'sequence': '어벤져스는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.017189430072903633,
  'sequence': '어벤져스는 정말 재미있는 드라마 다.',
  'token': 4665,
  'token_str': '드라마'},
 {'score': 0.014989713206887245,
  'sequence': '어벤져스는 정말 재미있는 이야기 다.',
  'token': 3758,
  'token_str': '이야기'},
 {'score': 0.009382630698382854,
  'sequence': '어벤져스는 정말 재미있는 장소 다.',
  'token': 4938,
  'token_str': '장소'}]

In [15]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.83824223279953,
  'sequence': '어벤져스는 정말 재미있는 영화 다.',
  'token': 3771,
  'token_str': '영화'},
 {'score': 0.02827564999461174,
  'sequence': '어벤져스는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.017189430072903633,
  'sequence': '어벤져스는 정말 재미있는 드라마 다.',
  'token': 4665,
  'token_str': '드라마'},
 {'score': 0.014989713206887245,
  'sequence': '어벤져스는 정말 재미있는 이야기 다.',
  'token': 3758,
  'token_str': '이야기'},
 {'score': 0.009382630698382854,
  'sequence': '어벤져스는 정말 재미있는 장소 다.',
  'token': 4938,
  'token_str': '장소'}]

### 5) 코랩에서 TPU 사용하기

In [1]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.27.8.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.27.8.210:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [2]:
# TPU Strategy 셋팅
# tf.distribute.Strategy는 훈련을 여러 GPU 또는 여러 장비, 여러 TPU로 나누어 처리하기 위한 텐서플로 API
# 이 API를 사용하면 기존의 모델이나 훈련 코드를 분산처리할 수 있음

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### 딥러닝 모델의 정의
* 딥러닝 모델을 정의할 때도 추가적인 코드 필요
* 모델 생성은 strategy.scope 내에서 이루어져야 함
  * 이는 모델을 정의하는 create_model()과 같은 함수를 만들어 strategy.scope 내에서 해당 함수를 호출하여 모델을 컴파일 하는 식으로 진행


In [3]:
# 다음과 같이 임의의 모델을 정의하는 create_model()이라는 함수 만듦

def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

In [5]:
# 이제 with strategy.scope(): 다음에 들여쓰기를 하고, create_model() 함수를 호출하고 모델을 컴파일
# 다시 말해 strategy.scope 내에서 모델이 생성되도록 함

with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam', 
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                metrics=['sparse_categorical_accuracy'])

# 이제 이 모델을 fit()하게 되면 해당 모델은 TPU를 사용하게 됨
# 딥러닝 모델의 정의 전까지 코드 실행 후, BERT와 커스텀 레이어에 대한 모델 정의와 컴파일을 strategy.scope 내에서 하면 됨

## 6-8까지는 비공개되었음
## 9) BERT의 문장 임베딩을 이용한 한국어 챗봇
* 사전 훈련된 BERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지인 sentence_transformers를 사용하여 쉽고 간단하게 한국어 챗봇 구현


In [6]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 3.1 MB 10.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.1 MB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 52.4 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=1aed6c8f530ab75d76ecfb6b8a003ac0f9164dd1a1aae75e0694b71a46528b42
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
# 문장 임베딩을 얻기 위해서 사전 훈련된 BERT 로드
# 여기서는 한국어도 포함되어 있는 학습 모델로 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# 데이터에서 모든 질문열에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [11]:
# 두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim 정의
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [14]:
# return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열
# 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하며 
# 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냄
# 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴

def return_answer(question):
  embedding = model.encode(question)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
  return train_data.loc[train_data['score'].idxmax()]['A']

In [15]:
return_answer('결혼하고싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [16]:
return_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [17]:
return_answer('반가워')

'저도 반가워요.'

In [18]:
return_answer('사랑해')

'상대방에게 전해보세요.'

In [19]:
return_answer('너는 누구니?')

'저는 위로봇입니다.'

In [20]:
return_answer('너무 짜증나')

'짜증날 땐 짜장면'

In [21]:
return_answer('화가납니다')

'화를 참는 연습을 해보세요.'

In [22]:
return_answer('나랑 놀자')

'지금 그러고 있어요.'

In [23]:
return_answer('나랑 게임하자')

'같이 놀아요.'

In [24]:
return_answer('출근하기싫어')

'씻고 푹 쉬세요.'

In [25]:
return_answer('여행가고싶다')

'이김에 떠나보세요.'

In [26]:
return_answer('너 말 잘한다')

'그런 사람이 있으면 저 좀 소개시켜주세요.'